In [17]:
import pandas as pd
path = '..\\data\\school_geolocations_with-connnectivity.csv'


In [79]:
class UnconnectedSchoolsExtractor:
    def __init__(self, path):
        self.path = path
        self.data = self._load_data(self.path)

    def _load_data(self, path):
        data = pd.read_csv(path)
        return data
    
    def get_unconnected_schools(self, country=None):
        output_columns = ['country', 'education_level', 'school_name', 'latitude', 'longitude']
        if country is None:
            return self.data[self.data['connectivity'] == 'No'].reset_index(drop=True)\
                [output_columns]
        else:
            return self.data[(self.data['country'] == country) & (self.data['connectivity'] == 'No')].reset_index(drop=True)\
                [output_columns]



In [ ]:
unconnected_schools = UnconnectedSchoolsExtractor(path)
unconnected_schools = unconnected_schools.get_unconnected_schools('Kenya')
display(unconnected_schools)

,country,education_level,school_name,latitude,longitude
0,Kenya,Secondary,SIAYA TOWNSHIP SECONDARY SCHOOL,0.066600,34.284000
1,Kenya,Secondary,TETU SUB COUNTY PRIVATE CENTRE,-0.476000,36.929001
2,Kenya,Primary,MWANGA,1.082841,35.182041
3,Kenya,Secondary,KISANGULA SECONDARY SCHOOL,0.079700,34.766998
4,Kenya,Secondary,KERUNDUT MIXED DAY SECONDARY SCHOOL,-0.778990,35.340939
...,...,...,...,...,...
6539,Kenya,Secondary,ST .NIMROD ACADEMY - KOMAHILL,-1.286344,37.120338
6540,Kenya,Secondary,KATHONZWENI GIRLS SECONDARY SCHOOL,-1.923272,37.678413
6541,Kenya,Secondary,KUNYAK SECONDARY SCHOOL,0.144890,35.291382
6542,Kenya,Secondary,GREMON EDUCATION CENTRE,-4.006350,39.704498


In [61]:
data[(data['country'] == 'Kenya') & (data['connectivity'] == 'No')]

,country,iso2_code,iso3_code,school_id_giga,school_name,admin1_id_giga,admin2_id_giga,education_level,connectivity,latitude,longitude,school_data_source
12297,Kenya,ke,ken,28bcee0b-fe95-3c36-881a-5b0d6bfc604d,SIAYA TOWNSHIP SECONDARY SCHOOL,KEN038,KEN038002,Secondary,No,0.066600,34.284000,NaN
12298,Kenya,ke,ken,28c04a96-dbb8-3c1d-9b77-5afae16ca68c,TETU SUB COUNTY PRIVATE CENTRE,KEN036,KEN036002,Secondary,No,-0.476000,36.929001,NaN
12299,Kenya,ke,ken,28cfdd40-8fb5-3ced-940a-26402778aa74,MWANGA,KEN042,KEN042004,Primary,No,1.082841,35.182041,NaN
12300,Kenya,ke,ken,28e1600b-b36f-32c5-91b8-f2840423d757,KISANGULA SECONDARY SCHOOL,KEN045,KEN045001,Secondary,No,0.079700,34.766998,NaN
12301,Kenya,ke,ken,28f61196-d3b4-3603-8662-822aa1c4ea12,KERUNDUT MIXED DAY SECONDARY SCHOOL,KEN002,KEN002001,Secondary,No,-0.778990,35.340939,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
65178,Kenya,ke,ken,ff9e58d2-a481-3e18-9fa6-9ea13de337ed,ST .NIMROD ACADEMY - KOMAHILL,KEN022,KEN022004,Secondary,No,-1.286344,37.120338,NaN
65179,Kenya,ke,ken,ffcc313b-d477-352d-838c-fc015c1ebbee,KATHONZWENI GIRLS SECONDARY SCHOOL,KEN023,KEN023003,Secondary,No,-1.923272,37.678413,NaN
65180,Kenya,ke,ken,ffd70a0e-29aa-35c5-a706-c72c656ead71,KUNYAK SECONDARY SCHOOL,KEN032,KEN032003,Secondary,No,0.144890,35.291382,NaN
65181,Kenya,ke,ken,ffdb97f4-d033-3f1e-b34b-352787a72a20,GREMON EDUCATION CENTRE,KEN028,KEN028006,Secondary,No,-4.006350,39.704498,NaN
